In [ ]:
#### 외국인현황 전처리 

hdfs dfs -put 서울시_외국인_현황.csv /user/engineer/

foreigner_seoul = spark.read.format('csv').option('header','true').load('서울시_외국인_현황.csv/part-00000-*.csv')

foreigner_seoul1 = foreigner_seoul.filter(col('지역코드')!='합계')

foreigner_seoul2 = foreigner_seoul1.filter(col('기간코드')!='2020')

foreigner_seoul_new=foreigner_seoul2.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/project/서울시_외국인.csv")

hdfs dfs -get /project/서울시_외국인.csv ./

In [ ]:
### 서울_유흥 전처리

seoul_alcohol = spark.read.format("csv").option("header","true").option("encoding","utf8").load("서울_유흥.csv")

seoul_open3 = seoul_open2.replace(["은평구","서대문구","마포구","양천구","강서구","구로구","금천구","영등포구","동작구","송파구","강동구","종로구","중구","용산구","성동구","광진구","동대문구","성북구","강북구","도봉구","관악구","서초구","강남구","중랑구","노원구"],["111181","111191","111201","111301","111212","111221","111281","111231","111241","111273","111274","111123","111121","111131","111142","111141","111152","111161","111291","111171","111251","111262","111261","111151","111311"],"지역구")

seoul_open4=seoul_open3.select(monotonically_increasing_id().alias("ID"),"*")

seoul_open5=seoul_open4.withColumnRenamed("지역구","지역코드")

seoul_open6 = seoul_open5.select("ID","지역코드","인허가일자","지번주소")

seoul_open7 = seoul_open6.filter(seoul_open.지번주소 != '-')

In [ ]:
##### 서울시 인허가일자 년도별로 자르기

seoul_open8 = seoul_open7.select(col('ID'),col('지역코드'), substring(seoul_open7.인허가일자, 1, 4).alias('기간'), col('인허가일자'), col('지번주소'))

In [ ]:
##### 년도를 기간코드로 바꿔주기

 seoul_open9= seoul_open8.replace(["2017","2018","2019"],["17","18","19"],"기간")

seoul_open8.where(col("기간") <= 2017).show(5)

 seoul_open9 = spark.sql("SELECT * FROM WHERE seoul_open8 (COL("기간") <= 2017"))

seoul_open9 = seoul_open8.groupBy('기간').agg(count('인허가일자'))

seoul_open10=seoul_open9.orderBy(col("기간").desc())

seoul_open9 =seoul_open8.filter(col('기간') !='1899')

seoul_open10 =seoul_open9.filter(col('기간') !='1904')

In [ ]:
 seoul_open11= seoul_open10.replace(['1967','1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017','2018','2019'],['17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17', '17','17','18','19'],"기간")

In [ ]:
 seoul_open12= seoul_open11.withColumnRenamed("기간","기간코드")

seoul_open_new=seoul_open12.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/project/서울시_유흥업소.csv")



In [ ]:
#### 서울시 유동인구 현황 전처리

people_seoul_new = spark.read.format('csv').option('header','true').load('서울시_유동인구_현황.csv/part-00000-*.csv')

In [ ]:
people_seoul1 = people_seoul.select(col("총생활인구수").alias("ad_id"),
                       col("part").alias("part_id"),
                       func.round(col("new_bid"), 2).alias("bid"))

In [ ]:


people_seoul_new1 = people_seoul_new.select("*",bround(col("총생활인구수"),0).alias("총생활인구수"),bround(col("일최대인구수"),0).alias("일최대인구수"),bround(col("일최소인구수"),0).alias("일최소인구수"),bround(col("주간인구수"),0).alias("주간인구수"),bround(col("야간인구수"),0).alias("야간인구수"),bround(col("일최대이동인구수"),0).alias("일최대이동인구수"),bround(col("자치구간이동인구수"),0).alias("자치구간이동인구수"))

In [ ]:
people_seoul_new1 = people_seoul_new.select("*",bround(col("총생활인구수")).alias("총생활인구수")